In [ ]:
HTML(read(open("style.css"), String))

# Play a Chess Game

In [ ]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random
# Pkg.add("NBInclude")
using NBInclude

## Import Notebooks

### Engines

In [ ]:
# @nbinclude("RandomChess.ipynb")

In [ ]:
# @nbinclude("Minimax.ipynb")

In [ ]:
# @nbinclude("AlphaBetaPruning.ipynb")

In [ ]:
@nbinclude("IterativeDeepening.ipynb")

### Utility

In [ ]:
@nbinclude("PGN_Export.ipynb")

In [ ]:
@nbinclude("PGN_Import.ipynb")

In [ ]:
# @nbinclude("EvaluatePosition.ipynb")

In [ ]:
@nbinclude("Utility.ipynb")

The function `playMove` is a helping function for the main `playGame` function and takes in the played game and it's current static evaluation. It applies an engine move to the game and information about the current evaluation of the engine, the static evaluation and the length of the cache. The function returns the new static evaluation as `current_boardscore` of the board after applying the engine move.

Possible engines are the `Random move chooser`, `Minimax Algorithm`, `Alpha-Beta-Pruning Algorithm` and `Iterative Deepening Algorithm`.

In [ ]:
function playMove(game::Game, aBoard::AdvBoard, cache::Dict{UInt64, Tuple{String, Int64, Int64}})
    starttime = time()
    #---------------------- Setup Engine ----------------------------
    # RANDOM ENGINE
    # eval, move = 0, generateRandomMove(game)
    
    # MINIMAX ENGINE
    # eval, move = minimax(board(game), currentcurrent_boardscorescore, 4, cache)

    # ALPHA BETA PRUNING ENGINE
    # eval, move = alphaBetaPruning(board(game), current_boardscore, 4, cache)

    # ITERATIVE DEEPENING ENGINE
    eval, move = iterativeDeepening(aBoard, 5, cache, true, 1.0, true)

    #----------------------------------------------------------------
    
    println("The Engine needs to be calculated the bestmove: $(time() - starttime) seconds")
    println("Current board score: $(aBoard.score)")
    println("Evaluation of engine: $eval")
    println("Current entries in Cache: $(length(cache))")
    println("Computer play: $move")
    clearCache(aBoard, move, cache)
    domove!(game, move)
    domoveAdv!(aBoard, move)
    cleanCacheBecauseRep!(aBoard, cache)
    return aBoard
end

## Main function

The function `playGame()` allows a user to play a chess game. The AI is set via the `playMove(game)` function. After the game is finished (checkmate or draw) the game is saved as a PGN-file and saved in the /Games directory.

The function asks the user to input their name and what color they want to play as at the beginning. After that the player and the engine take alternate moves playing a chess game. 

Moves are inputted as strings which contain the pieces current location and the pieces end location. For example playing `1. e4` as white requires the string `e2e4` as input.

To resign the user can type `resign` as his move.

In [ ]:
function playGame(pgnFile=nothing)
    # Setup Board
    if pgnFile == nothing
        # new Game
        println("Initializing new Game...")
        game = Game()
        setGameHeaders(game)
    else
        # continue saved game
        println("Opening $(pgnFile) ...")
        game = openPGNtoGame(pgnFile)
        if game == nothing
            println("No such file found")
            return
        end
        toend!(game)
        println(game)
    end
    
    aBoard = AdvBoard(board(deepcopy(game)))
    result = ""
    
  
    println("What's your name?")
    username = readline()
    
    println("Do you want to play as 'white' or 'black'?")
    color = readline()
    color = lowercase(color)
    # Offset move if player is black
    cache = initCache()
    if color != "black" && color != "white"
        print("Invalid color. Choose white or black")   
        return
    elseif color  == "black"
        setheadervalue!(game, "Black", username)
        aBoard = playMove(game, aBoard, cache)
        printGame(game)
    elseif color == "white"
        setheadervalue!(game, "White", username)
    end
    while true
        # User Move
        if !isterminal(game)
            println("Make a move. Other actions: 'resign' or 'pause'")
            userInput = readline()
            # Special user input handler
            if(userInput == "resign")
                println("You resigned the game. The engine wins.")
                result = string(sidetomove(board(game)))
                break
            elseif(userInput == "pause")
                println("Saving game to resume later")
                saveGameToPGN(game)
                return game
            end
            
            userMove = movefromstring(userInput)
            if(userMove == nothing)
                println("Invalid User Input. Please use the format [source] [destination]. Example(e2e4)")
                continue
            end
            if(!is_legal_move(game, userMove))
                println("Invalid Chess move. Please make a other move.")
                continue
            else
                clearCache(aBoard, userMove, cache)
                domove!(game, userMove)
                domoveAdv!(aBoard, userMove)
                # cleanCacheBecauseRep!(aBoard, cache)
            end
            printGame(game)
        else
            break
        end
        
        # Engine Move
        if !isterminal(game)
            aBoard = playMove(game, aBoard, cache)
            printGame(game)
        else
            break
        end
    end
    setWin(game, result)
    println(game)
    saveGameToPGN(game)
    return game
end

In [ ]:
playGame()